## Advanced LEGO assortment 

In this notebook, we will continue building from where we left off with the optimization model defined in <a href="$./Optimization_Model">Optimization_Model</a>. In the previous notebook, you can find a step by step, detailed description of a mathematical optimization model for our mathematical programming assortment problem and its resulting optimization. This notebook will continue progressing toward more advanced modelling and utilize production-ready code relying on object oriented programming techniques.

If necessary, you can rebuild the LEGO brick inventory in <a href="$./Prepare_Data">Prepare_Data</a>. The resulting catalog datasets with suffix `_large` define the new inventory of parts from a collection of more than 200 Star Wars-themed LEGO sets released since the year 2000, which will be used in this notebook.

We will create a class object with the same components (objective, variables, constraints) as our model in <a href="$./Optimization_Model">Optimization_Model</a>, and solve an instance of this base model class, parameterized by the larger brick inventory.

To illustrate the value of optimization we will extend our experiment with:
- A comparison to a greedy search heuristic that does not use a mathematical programming optimization approach: This showcases the value gained from mathematical optimization
- Solution for a variation of this problem where we allow for brick parts to be purchased: This showcases how mathematical optimization solutions can be easily extended

## Set up and initialize

### Import necessary packages

The optimization model will be implemented using Gurobipy (Python API of Gurobi).  We need to make sure that Gurobipy is installed and ready to be used.

In [0]:
%pip install gurobipy==12.0.0
dbutils.library.restartPython()

In [0]:
# Gurobi optimization library for model implementation
from gurobipy import GRB 
import gurobipy as gp

# Importing essential modules from pyspark.sql for DataFrame and Spark session management
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np

# Experiment logging
import mlflow

# Filter warnings
import warnings
from pyspark.pandas.utils import PandasAPIOnSparkAdviceWarning
warnings.filterwarnings('ignore', category=PandasAPIOnSparkAdviceWarning)

### Load input data from Catalog

With our imports set, the next step is to load the data required for the model. We'll use the data previously processed into tables in the Prepare_Data notebook as inputs.

In [0]:
# Create a widget to accept a catalog name
dbutils.widgets.text('catalog_name','catalog_name','Enter catalog name')
dbutils.widgets.text('schema_name','schema_name','Enter schema name')

In [0]:
# Retrieve the catalog name
catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
print(catalog_name + "/" + schema_name)

### Gurobi license and settings

Unlike the small scale example in our initial example with 4 LEGO sets, the size of the problem is now parameterized by the larger inventory requires a commercial Gurobi licence. Here we set up the license with a [Gurobi Cluster Manager](https://docs.gurobi.com/projects/remoteservices/en/current/index.html) and store the API details using [Databricks Secrets](https://docs.databricks.com/en/security/secrets/index.html).

To obtain a license and discuss alternatives to this licensing set up, contact [Gurobi Sales](mailto:sales@gurobi.com).

In [0]:
GUROBI_LIC = {
    "CSMANAGER": "https://gcm.aimpointdigital.com",
    "CSAPIACCESSID": dbutils.secrets.get(scope="gurobi_lic_jm", key= "CSAPIACCESSID"),
    "CSAPISECRET": dbutils.secrets.get(scope = "gurobi_lic_jm", key = "CSAPISECRET"),
    "CSAPPNAME": dbutils.secrets.get(scope = "gurobi_lic_jm", key = "CSAPPNAME")
}

GUROBI_OPTIONS = {
    "TimeLimit": 60,  # Sets a time limit for the optimization process (in seconds).
    "MIPGap": 1e-3,   # Specifies the relative optimality gap.
    "Threads": 8      # Limits the number of threads Gurobi uses for computation to 8.
}

## Base LEGO Assortment Model Class

We will define a class object of the base LEGO assortment model. For a step-by-step review of the mathematical programming concepts and a detailed description of the base model please refer to the notebook <a href="$./Optimization_Model">Optimization_Model</a>. The description for each of the elements of the optimization is also given in the doc string for each component defined within the `BaseLegoModel` class. Using this type of definition for our base model is beneficial because it promotes code reusability and modularity. By defining a base model class, we can extend its functionality in derived classes through class inheritance without duplicating code. This approach ensures that any updates or changes to the base model are automatically inherited by all derived classes, making the code easier to maintain and extend. Additionally, it allows us to implement new features, such as the parts purchase logic (implemented towards the end of this notebook), in a structured and consistent manner.

The class defines all the components required to formulate and solve the assortment optimization problem:
- The sets and parameters that describe the available inventory as generated by the data preparation notebook that runs in the first code cell of this notebook
- An instance of a gurobi environment class `Env` parameterized by license arguments in `GUROBI_LIC` which will house the models to be solved by Gurobi
- An instance of a gurobi model class `Model` parameterized initially with the solver parameters in `GUROBI_OPTIONS` which will be attached to the environment and modified with the mathematical program components:
    - `_binary_variables` defines the set of binary variables (0 or 1) indexed over LEGO sets that indicate whether or not a LEGO set is built in the optimized collection
    - `_constraint_part_inventory`defines the set of constraints that assure that the inventory quantity for each part in a particular color is respected. Namely, we cannot use more parts than available to build LEGO sets. Ultimately these constraints force us to choose the most valuable sets overall subject to inventory availability
    - `_objective_function` defines an objective function expression that specifies the total value of the collection by summing the individual contributions from the optimally selected sets. In other words, no value is contributed to the objective function from a set that is not selected to be built. In this case we want to obtain the highest value possible which corresponds to a maximization of this objective function.
    
- `define_model` builds the model object with all the required components
- `solve_model` solves the model object in the specified gurobipy environment
- `write_solution` writes the set selection variables and unused inventory parts to dataframes
- `free_environment`  first disposes the model attached to the environment, and subsequently disposes the environment freeing up the license to be used for other applications

In [0]:
class BaseLegoModel():
    """ LEGO assortment optimization base model.
    """
    def __init__(
        self, 
        s_lego_set: set, 
        s_part_in_set: set,
        s_part_in_color: set,
        p_part_quantity_in_set: dict, 
        p_part_quantity_available: dict,
        p_set_num_value: dict
    ) -> None:
        """ Initialize model instance.
        
        Defines the gurobipy environment and model objects, with the licensing and solver options
        in GUROBI_LIC and GUROBI_OPTIONS.

        Reads the sets and parameter arguments as attributes to the class on initialization.
    
        Parameters:
        
        - s_lego_set: set of (set_num) values. Each 'set_num' corresponds to a unique LEGO set that we will attempt to make 
                      with available inventory
        - s_part_in_set: set of (set_num, part_num, color_id) tuples indicating which parts and colors are included in each 
                         buildable LEGO set
        - s_part_in_color: set of (part_num, color_id) tuples defining specific parts with their associated colors
                           that constitute our inventory
        - p_part_quantity_in_set: Dictionary defining number of parts of specified type and color required to build a LEGO set.
                                  Dictionary keys are (set_num, part_num, color_id) tuples
                                  Dictionary values are the number of parts required of color `color_id` and type `part_num` in set `set_num`
        - p_part_quantity_available: Dictionary defining parameters for quantities of parts available in inventory
                                     Dictionary keys are (part_num, color_id) tuples
                                     Dictionary values are the amount of parts of type `part_num` and color `color_id` in inventory
        - p_set_num_value: Dictionary defining parameters for value of each buildable LEGO set
                           Dictionary keys are (part_num)
                           Dictionary values are the numerical value assigned to building a complete set defined by `set_num`
                           
        """
        
        # Optimization sets & parameters
        self.s_lego_set = s_lego_set
        self.s_part_in_set = s_part_in_set
        self.s_part_in_color = s_part_in_color
        self.p_part_quantity_in_set = p_part_quantity_in_set
        self.p_part_quantity_available = p_part_quantity_available
        self.p_set_num_value = p_set_num_value

        # Set the Gurobi license parameters and initialize the environment
        self.env = gp.Env(empty=True)
        # This line creates an empty Gurobi environment. By using empty=True, the environment is initialized without automatically loading parameters or settings, allowing for explicit customization before starting it.

        for param_name, param_val in GUROBI_LIC.items():
            self.env.setParam(param_name, param_val) # This line specifies the license parameters for the Gurobi environment.
        self.env.start()
        #This line starts the environment, activating the connection between the optimization program and the Gurobi solver. This step is essential after creating an empty environment to ensure it is ready to use.

        # Set the Gurobi solver options and initialize the model
        self.model = gp.Model("base_lego_problem", env=self.env)
        for param_name, param_val in GUROBI_OPTIONS.items():
            self.model.setParam(param_name, param_val)  # This line specifies the other parameters that might be specified for the Gurobi solve, such as time limits or optimality gaps
    
    def _binary_variables(self) -> None:
        """ Define the binary optimization variables for LEGO set selection.

        There is a binary variable for each buildable LEGO set s in the set of buildable LEGO sets.
        It takes a value of 1 if the set is selected to be built from inventory, and 0 otherwise.        
        """
        self.bv_select_set = self.model.addVars(
            self.s_lego_set,         # the set of elements that the decision variable is indexed over
            vtype=GRB.BINARY,        # specifies that these variables are binary (0 or 1)
            name="bv_select_set"     # define a name for the variable
        )
    
    def _constraint_part_inventory(self) -> None:
        """ Define the constraint that ensures that each part-color is
        used according to inventory availability.

        This is a balance constraint that ensures that the parts used across all the sets
        are within the inventory limits. This constraint is defined for each part-color combination.
        The left hand side is the sum of the parts used in the sets, and the right hand side is the
        inventory limit for that part-color combination.
        """
        # Iterate over all part-color combinations
        for (p, c) in self.s_part_in_color:
            cons_name = f"c_part_quantity_balance[{p}-{c}]"
            constr = (
                sum(   # Define the left-hand side (LHS) of the constraint: Sum of the quantities of part (p) in color (c) across all selected sets (s).
                    self.p_part_quantity_in_set[s, p, c] * self.bv_select_set[s]  # Quantity of (p, c) in set (s) times the binary variable
                    for s in self.s_lego_set
                    if (s, p, c) in self.s_part_in_set  # Only consider sets that actually contain the part-color combination
                ) <= self.p_part_quantity_available[p, c]
            )
             # Add the constraint to the model with the generated name
            self.model.addConstr(constr, name=cons_name)
    
    def _objective_function(self) -> None:
        """ Define the objective function for the LEGO set selection problem as the sum of values
        for sets selected to be built from inventory.

        Only the sets selected to be built from inventory are included in the result since for
        those that aren't, the binary variable bv_select_set takes value of 0.

        We are interested in obtaining the most valuable collection of LEGO sets, so the
        objective function is set to be maximized.
        """
        obj = sum(self.p_set_num_value[s] * self.bv_select_set[s] for s in s_lego_set)
        self.model.setObjective(obj, GRB.MAXIMIZE)
    
    def define_model(self) -> None:
        """ Define the model for the LEGO set selection problem with the specified variables,
        constraints, and objective.

        Calling self.model.update() is required to include the newly defined variables,
        constraints, and objective function.
        """
        self._binary_variables()
        self._constraint_part_inventory()
        self._objective_function()
        self.model.update()

    def solve_model(self) -> None:
        """ Solve the specified model with Gurobi and store the optimized value.
        
        Returns an exception if model is infeasible.
        """
        self.model.optimize()

        # Check the model's status to ensure that an optimal solution was found.
        if self.model.status == GRB.OPTIMAL:
            # If the model status indicates an optimal solution was found, display a success message and store the objective value
            print("Optimal solution found")
            self.optimal_value = self.model.ObjVal                    
        else:
            # If the model status indicates that the solution is not optimal (e.g., infeasible), raise an exception.
            raise Exception("Base model is infeasible.")
    
    def write_solution(self) -> None:
        """ Write the solution selected sets and unused inventory parts to DataFrame objects"""
        
        # retrieves the solution values for the variables in bv_select_set from the Gurobi model
        bv_select_data = {
            var: val
            for var, val in self.model.getAttr('X', self.bv_select_set).items()
        }

        # since our decision variable is a binary variable, we just need to keep the sets that are selected by the model (i.e. 
        # if the result is 1). Please note that we use a tolerance to avoid floating point precision issues when filtering.
        self.df_bv_select_data = pd.DataFrame(
            [
                {'Set': k, 'Value': v}
                for k, v in bv_select_data.items()
                if v > 0.01 # We are looking for elements where the value is 1 (not 0). We use a numerical tolerance to avoid floating point precision issues
            ]
        )
        
        # Remaining parts
        # We start with parts available and we subtract parts that are used up in building selected sets
        remaining_parts = {
            (p, c): base_model.p_part_quantity_available[p, c] - sum(
                base_model.p_part_quantity_in_set[s, p, c] * bv_select_data[s]
                for s in base_model.s_lego_set
                if (s, p, c) in base_model.s_part_in_set
            )
            for (p, c) in base_model.s_part_in_color
        }
        
        # Create a dictionary of remaining parts where there are nonzero values available, in other words where there is actual inventory
        remaining_parts_data = {
            (p, c): remaining_parts[p, c]
            for (p ,c) in self.s_part_in_color
            if remaining_parts[p, c] > 0
        }

        self.df_remaining_parts_data = pd.DataFrame(
            [
                {'Part': k[0], 'Color': k[1], 'Quantity': v}
                for k, v in remaining_parts_data.items()
            ]
        )

    def free_environment(self) -> None:
        #Best practice is to release the Gurobi environment when you are done utilizing it.
        """ Free the Gurobi environment."""
        self.model.dispose()
        self.env.dispose()


### Load data from Catalog for Model Parameterization

In order to fully specify our optimization model, we need to provide all the required parameters stored in the catalog specified by the `catalog_name` widget found at the top of this notebook. The datasets are stored in your data catalog by the data preparation <a href="$./Prepare_Data">Prepare_Data notebook</a> in the first cell. For this notebook, relevant datasets include a `_large` suffix.

In [0]:

# Read in all the necessary data tables
relevant_set = ps.read_table(f"{catalog_name}.{schema_name}.relevant_set_large")
lego_sets = ps.read_table(f"{catalog_name}.{schema_name}.sets_large")
parts = ps.read_table(f"{catalog_name}.{schema_name}.parts_large")
colors = ps.read_table(f"{catalog_name}.{schema_name}.colors_large")
parts_avail = ps.read_table(f"{catalog_name}.{schema_name}.parts_avail_large")
part_in_set = ps.read_table(f"{catalog_name}.{schema_name}.part_in_set_large")

Next, we transform the raw datasets into sets that define buildable LEGO sets, brick parts, colors, and build requirements and can be used as input arguments to the `BaseLegoModel` class. These are the building blocks (pun intended) of your project. Once you have these parameters, it’s time for model building and **optimization**.

In [0]:
## Helper function to convert a pandas-on-Spark DataFrame to a dictionary
def ps_to_dict(df_ps, column_key, column_value):
    """
    Converts a pandas-on-Spark DataFrame to a dictionary.
    
    Parameters:
    df_ps (ps.DataFrame): The pandas-on-Spark DataFrame.
    column_key (str or list of str): Column(s) to use as the dictionary key(s).
    column_value (str): Column to use as the dictionary value.
    
    Returns:
    dict: A dictionary where keys are derived from `column_key` and values from `column_value`.
    """

    # Single column key
    if isinstance(column_key, str):
        dictionary = dict(zip(
            df_ps[column_key].to_numpy(), 
            df_ps[column_value].to_numpy()
            ))

    # Multi-column key
    elif isinstance(column_key, list):
        
        dictionary = dict(zip(
            zip(*(df_ps[col].to_numpy() for col in column_key)),
            df_ps[column_value].to_numpy()
        ))
    
    else:
        raise ValueError("column_key must be a string or a list of strings.")
    
    return dictionary


## Create sets for the optimization model from the data

# Create a set of unique part numbers 
s_part = set(
    parts['part_num'].to_numpy()
)
             
# Create a set of unique LEGO set numbers 
s_lego_set = set(
    lego_sets['set_num'].to_numpy()
)

# Create a set of unique color IDs 
s_color = set(
    colors['id'].to_numpy()
)

# Create a set of (part_num, color_id) pairs representing parts and their associated colors
s_part_in_color = set(
    zip(
        part_in_set['part_num'].to_numpy(), 
        part_in_set['color_id'].to_numpy()
    )
)


# Create a set of (set_num, part_num, color_id) tuples indicating which parts and colors are included in each set
s_part_in_set = set(
    zip(
        part_in_set['set_num'].to_numpy(),
        part_in_set['part_num'].to_numpy(),
        part_in_set['color_id'].to_numpy()
    )
)

## Define the parameters for the optimization model

# number of parts available
p_part_quantity_available = ps_to_dict(
    parts_avail,
    ["part_num", "color_id"],
    "quantity"
)

# number of parts in each LEGO set per specific part type
p_part_quantity_in_set = ps_to_dict(
    part_in_set,
    ["set_num", "part_num", "color_id"],
    "quantity"
)

# part parameters
p_part_name = ps_to_dict(
    parts,
    'part_num',
    'name'
)
    
# LEGO set parameters 
p_set_num_part = ps_to_dict(
    lego_sets,
    ['set_num', 'year'],
    'num_parts'
)     

# set the quantity of all other parts to zero 
temp_dict = {
    (p, c):0
        for (p, c) in s_part_in_color
        if (p, c) not in p_part_quantity_available
}
p_part_quantity_available.update(temp_dict)

%md
#### Objective Function
For this example, we assume that the value of a set is proportional to the number of parts in the set. Thus, a set that has more parts is considered to be more desirable. Additionally, we assume older parts are more valuable

In [0]:
# value of each set in the objective
## assume set value is proportional to the number of parts in the set and higher for older sets
year_gain_factor = 1.1
p_set_num_value = {key[0]: value*year_gain_factor**(2025-key[1]) for key, value in p_set_num_part.items()}

We have all the input parameters needed in the LEGO Assortment Optimization Model. 

### Model Tracking
Throughout the experimentation below we will demonstrate how to store important parameters and results using [MLflow Tracking](https://www.mlflow.org/). This ensures that our analysis is well-documented and easily accessible for future reference.

### Build & Solve Model

The inventory is parameterized in the sets and parameters that serve as the input arguments to the model class. The solve_model method will call Gurobi to explore the feasible space and build the most valuable collection of complete sets based on the perturbed inventory from the parts corresponding to more than 200 starting Star Wars LEGO sets.

The output log of the solution progress will be returned as the solver works to find the optimal solution.

In [0]:
with mlflow.start_run(run_name="BaseLegoModel Large Inventory") as run:
    # We are creating an instance of the BaseLegoModel called base_model
    # that is comprised of the various data components s_lego_set, s_part_in_set, etc.
    base_model = BaseLegoModel(
        s_lego_set,
        s_part_in_set,
        s_part_in_color,
        p_part_quantity_in_set,
        p_part_quantity_available,
        p_set_num_value
    )

    # We are now calling the functions defined within the BaseLegoModel to generate, solve, and write the solution for the base_model
    base_model.define_model()
    base_model.solve_model()
    base_model.write_solution()

    # Log parameters and metrics
    mlflow.log_metric("optimal_value", base_model.optimal_value)
    mlflow.log_metric("total_inventory_unused_bricks", base_model.df_remaining_parts_data.Quantity.sum())

As shown by the solver output above, Gurobi finds the optimal solution after a few iterations for the larger LEGO assortment problem definition. Starting with a heuristic solution with an objective surrounding 80k, the solver progresses to a final solution that is slightly higher than 95k and a gap of 0%, which means global optimality has been achieved. In other words, the solver guarantees that a better solution does not exist.

## Greedy Search Comparison
Let’s explore an alternative solution that skips the optimization solver to get a "good enough" solution. We implement a greedy search algorithm where we attempt to build LEGO sets with the highest value as much as possible. First, we sort LEGO sets in descending order of value. Then, we attempt to build the most valuable set first. If successful, the parts are removed from the available inventory, and the algorithm moves to the next valuable set in the queue to determine if it can be built or not. This process continues until all sets have been considered or all available inventory is used up.

In [0]:
with mlflow.start_run(run_name="GreedySearch Large Inventory") as run:
    # First assign value consistent to data preparation and order the lego sets in descending value
    lego_sets['value'] = lego_sets['num_parts']*year_gain_factor**(2025-lego_sets['year'])
    lego_sets = lego_sets.sort_values('value', ascending=False).reset_index(drop=True)
    
    # Next, we make a separate version of the available inventory for the heuristic to use so that we can compare solutions more easily
    greedy_search_inventory = p_part_quantity_available.copy()

    # Iterate over the ordered list to check if the most valuable LEGO sets can be built, and if so, remove the necessary parts from inventory
    greedy_search_sets_built = dict()
    for index, row in lego_sets.iterrows():
        # Check if the set identified by row['set_num'] can be built
        set_buildable = min( 
            {
                (p, c): (greedy_search_inventory[p, c] - # From available inventory, subtract
                        p_part_quantity_in_set[row['set_num'], p, c]) # parts required to build the set `set_num` 
                    if (row['set_num'], p, c) in s_part_in_set
                    else greedy_search_inventory[p, c]
                    for (p, c) in s_part_in_color
            }.values()
        ) >= 0 # Assuming we have sufficient parts to build the LEGO set (remaining are positive), we consider the set buildable
        if set_buildable:
            # If set is buildable, we update inventory to account for our decision to build this set
            greedy_search_sets_built[row['set_num'], row['name']] = row['value']
            # Update inventory to account for the fact that this LEGO set is now built, the parts are allocated and therefore not available to any other LEGO sets
            greedy_search_inventory = {
                (p, c): greedy_search_inventory[p, c] - p_part_quantity_in_set[row['set_num'], p, c]
                    if (row['set_num'], p, c) in s_part_in_set
                    else greedy_search_inventory[p, c]
                    for (p, c) in s_part_in_color
            }
    
    mlflow.log_metric("greedy_value", sum(greedy_search_sets_built.values()))
    mlflow.log_metric("total_inventory_unused_bricks", sum(greedy_search_inventory.values()))

In [0]:
# Compare to optimal solution
optimal_value = base_model.optimal_value
greedy_value = sum(greedy_search_sets_built.values())

optimal_unused_total = int(base_model.df_remaining_parts_data.Quantity.sum())
greedy_unused_total = sum(greedy_search_inventory.values())

print(f'''
      Optimal Collection Value:\t {round(optimal_value, 2)}
      Greedy Collection Value:\t {round(greedy_value, 2)}
      -----------------------------------------
      Difference:\t\t {round(optimal_value - greedy_value, 2)}
      Improvement:\t\t {round((optimal_value - greedy_value) / greedy_value * 100, 1)}%
      '''
)

print(f'''
      Optimal Solution Number of Unused bricks:\t {optimal_unused_total}
      Greedy Solution Number of Unused bricks:\t {greedy_unused_total}
      -----------------------------------------
      Difference:\t\t {greedy_unused_total - optimal_unused_total} (greedy - optimal)
      Improvement:\t\t {round((greedy_unused_total - optimal_unused_total) / greedy_unused_total * 100, 1)}%
      '''
)

The total value of the Heuristic-resulting LEGO set collection is 6.5% lower compared to the mathematical optimal solution. This value is achieved by reducing the total number of unused parts in the inventory by 10.1%, 2154 individual pieces. This highlights the benefits of mathematical optimization: you are able to achieve a higher total value with more efficient use of resources due to the holistic view of the problem at hand as opposed to the narrow scope utilized by a greedy heuristic.

An increased margin of more than 5% can yield enormous benefits for most real-world organizations that face this type of problem. The value of optimization becomes even more significant as the number of variables and constraints grows to millions, a scenario that is quite common in real-life assortment applications and makes the use of heuristics or rules-of-thumb suboptimal.

### Optimal vs. Greedy Search Built Collection Differences
Let’s take a closer look at how the collection built by the optimal approach achieves a higher value. First, we combine the lists of sets built by both approaches to see the differences between the solutions. Next, we perform aggregations to quantify their similarities and differences in terms of the number of sets built and their value. A table for the sets built by either approach in displayed in descending value. This analysis will clearly illustrate how the optimal strategy outperforms the greedy heuristic.

In [0]:
# Create a DataFrame of LEGO sets obtained from Greedy search
greedy_df = pd.DataFrame(
    data=greedy_search_sets_built.values(),
    index=greedy_search_sets_built.keys(),
    columns=['Value']
).reset_index()
greedy_df.columns = ['set_num', 'name', 'value']

# Create a DataFrame of LEGO sets obtained from optimization model
optimal_df = base_model.df_bv_select_data[['Set']].merge(
    lego_sets.to_pandas()[['set_num', 'name', 'value']],
    left_on='Set',
    right_on='set_num',
    how='left'
)
optimal_df.drop(columns=['Set'], inplace=True)

# We identify the sets built by each approach and identify:
# Sets built uniquely by Optimization Strategy
# Sets built uniquely by the Greedy Search Strategy
# Sets common to both strategies
combined_df = optimal_df.merge(greedy_df, how='outer', indicator=True)
combined_df.rename(columns={'_merge': 'Built by Strategy'}, inplace=True)
combined_df['Built by Strategy'].replace({'both': 'Both', 'left_only': 'Optimal Only', 'right_only': 'Greedy Only'}, inplace=True)
combined_df['Value Tercile'] = pd.qcut(combined_df['value'], 3, labels=['Low', 'Medium', 'High'])
display(combined_df.sort_values(by='value', ascending=False))
print('Number of sets built by each approach:')
display(combined_df['Built by Strategy'].value_counts())

### The Value of Optimization

Now that both the optimal and greedy solutions are available, we can compare them by aggregating and visualizing the obtained value from each method. We do this by labeling each set, built by either method, in value terciles (Low, Medium, High).

We then plot the distribution in number and value for each solution. To identify the specific differences we group the bar plots by the sets that are found in the solutions of both methods 'Both' and we label the ones found exclusively in each solution as '(Optimal or Greedy) Only'.

In [0]:
import matplotlib.pyplot as plt
_, ax = plt.subplots(1, 2, figsize=(16, 4))

combined_value_df = combined_df\
    .sort_values(by='Built by Strategy')\
    .pivot_table(values='value', index='Built by Strategy', columns='Value Tercile', aggfunc='sum')

combined_number_df = combined_df\
    .groupby(['Built by Strategy', 'Value Tercile'])['set_num']\
    .nunique()\
    .unstack()

combined_value_df.plot(kind='bar', stacked=True, color=['tab:blue', 'tab:orange', 'tab:green'], ax=ax[0])
for container in ax[0].containers:
    ax[0].bar_label(container, fmt='%.0f', label_type='edge', fontsize=7)
ax[0].grid(axis='y')
ax[0].set_xticklabels(['Optimal Only', 'Greedy Only', 'Both'], rotation=0)
ax[0].set_title('Value Contribution by Set Value Tercile and Strategy')

combined_number_df.plot(kind='bar', stacked=True, color=['tab:blue', 'tab:orange', 'tab:green'], ax=ax[1])
for container in ax[1].containers:
    ax[1].bar_label(container, fmt='%.0f', label_type='edge', fontsize=7)
ax[1].grid(axis='y')
ax[1].set_xticklabels(['Optimal Only', 'Greedy Only', 'Both'], rotation=0)
ax[1].set_title('Number of Built Sets by Set Value Tercile and Strategy')

In [0]:
# Recall, best practices are to free the gurobi environment after use
# Dispose base model instance
base_model.model.dispose()
base_model.env.dispose()

By having a closer look to the results of the optimization and greedy search approaches, the value of the latter is unveiled. Notice that the total number of sets in the optimal collection is much higher than the greedy search (110 vs. 92).

Fundamentally, the greedy approach is limited by not having a holistic view of the problem at hand. This can be checked by ordering the combined built set data by value. One can see that both approaches select the most valuable 13 sets. Further down the list, it is evident that the greedy approach is consistent with its name as valuable selections such as "Droid Gunship" or "Ahsoka's Starfighter and Vulture Droid" are made, which in turn make other less valuable sets impossible to build, reducing the overall value of the final collection.

In contrast, the optimization model chooses to make sets that might have lower value compared to sets such as "Droid Gunship", but by utilizing available parts efficiently, it is able to make many more of these lower value sets resulting in a higher overall objective value. 

## Extracting Additional Value

The value of optimization has been clearly demonstrated with the example above. Although the problem was increased in size significantly with respect to the minimal introductory problem instance, this is still very small and simple compared to real life Decision Science problems. When the number of variables and constraints grow large and complex enough, optimization could be the only efficient approach to obtain an economically feasible solution.

Below we will introduce interesting modifications to our LEGO assortment problem to illustrate how to translate real assortment considerations into efficient constraint formulations.

We start by examining the value of the collection that would have been obtained without perturbing the available inventory:

In [0]:
original_value = round(lego_sets['value'].sum(), 2)
print(f'''
      Unperturbed Collection Value:\t\t\t {original_value}
      Optimal (Perturbed Inventory) Collection Value:\t {round(optimal_value, 2)}
      -----------------------------------------
      Difference:\t\t\t\t\t {round(original_value - optimal_value, 2)}
      Loss Value:\t\t\t\t\t {round((original_value - optimal_value) / original_value * 100, 1)}%
      '''
)

Although the optimal solution extracted significant value from the perturbed inventory, there are still unused pieces and value left on the table in terms of sets that could have been built if no pieces were lost. 

One option to extract value from remaining unused parts is to selectively purchase specific pieces that will allow us to complete additional sets.

We introduce a "budget" of a finite number of parts to be purchased and let the optimization problem decide which are the best pieces to acquire with the goal of completing high value LEGO sets.

%md
### Part Purchasing Model Formulation

Now we are interested in extending the model logic to include the option of purchasing parts under a limited budget. The original formulation lacks the variables and expressions to handle this extension.

From the perspective of optimization model components we need two major changes:

- First, we introduce new variables that specify the quantity of each part in color that the optimization will select to purchase. We can choose to purchase multiple parts of the same type and color. Therefore, these variables are defined as 'integer variables' as they can take integral values (0, 1, 2, 3..)

    $$ iv^{\text{Purchase}}_{p,c} $$

    For example, the optimizer can determine that we should purchase 5 parts of type p in color c. The optimizer will do so to enable us to build additional collections which must be accounted for in the balance constraint.


- The original constraint needs to be modified to incorporate potential additional pieces obtained via purchase. We include an additional term in the right hand side of the original constraint expression. This term is exactly the 'integer variable' defined in the previous step and ensures newly purchased pieces if any, are correctly accounted for in our available inventory.

    $$ \sum_{s:(s,p,c)\in \text{PartInSet}} \text{PartQinSet}_{s,p,c} \times bv^{\text{SelectSet}}_{s} \leq \text{PartQAvail}_{p,c} + iv^{\text{Purchase}}_{p,c} \qquad \forall (p,c) \in \text{PartInColor}$$ 


- We assign a finite budget on the number of parts that can be purchased:

$$ \sum_{(p,c)\in \text{PartInColor}} iv^{\text{Purchase}}_{p,c} \leq \text{PurchaseLimit} $$


- Lastly, purchasing additional parts incurs a cost. We modify our objective function to correctly reflect this cost. For the purpose of this demonstration, we'll assume that all parts incur the same cost. The objective becomes the difference of the value of the collections completed and the cost of parts purchased:

$$ \text{max} \sum_{s} \text{SetValue}_{s} \times bv^{\text{SelectSet}}_{s} - \sum_{p,c} \text{UnitPartCost} \times iv^{\text{Purchase}}_{p,c}$$


We rely on class inheritance to avoid redefining methods that do not need to be updated and reuse parts of our existing implementation.


In [0]:
# Define a modified model class to allow part purchases
class PartPurchaseLegoModel(BaseLegoModel):
    """ LEGO assortment optimization model with part purchase
        constraints.  
    """

    def __init__(
        self, 
        s_lego_set: set, 
        s_part_in_set: set,
        s_part_in_color: set,
        p_part_quantity_in_set: dict, 
        p_part_quantity_available: dict,
        p_set_num_value: dict,
        p_max_parts_purchased: int,
        p_part_cost: int
    ) -> None:
        """ Initialize model instance with additional part budget parameters.

        First initialize the base model class. Two additional parameters are introduced.

        Parameters:
        - p_max_parts_purchased: int, maximum number of parts to be purchased
        - p_part_cost: int, cost of purchasing a single part (same for all parts)
        """
        # call the parent objects initialization method
        super().__init__(
            s_lego_set,
            s_part_in_set,
            s_part_in_color,
            p_part_quantity_in_set,
            p_part_quantity_available,
            p_set_num_value
        )
        self.p_max_parts_purchased = p_max_parts_purchased
        self.p_part_cost = p_part_cost
    
    def _integer_variables(self) -> None:
        """ Define the integer optimization variables that specify the number of a
        part (and its color) to be purchased.
        """
        self.iv_purchase = self.model.addVars(
            s_part_in_color,
            vtype=GRB.INTEGER,
            name="iv_purchase"
        )
    
    def _constraint_part_inventory(self) -> None:
        """ Modified part inventory balance constraint.
        
        The left hand side remains the same as the base model. The right hand side
        is modified to include the number of parts purchased for each part-color
        combination.
        """
        # Iterate over all part-color combinations
        for (p, c) in self.s_part_in_color:
            cons_name = f"c_part_quantity_balance[{p}-{c}]"
            constr = (
                sum(
                    self.p_part_quantity_in_set[s, p, c] * self.bv_select_set[s]
                    for s in self.s_lego_set
                    if (s, p, c) in self.s_part_in_set
                ) <= self.p_part_quantity_available[p, c] + self.iv_purchase[p, c]
            )
            self.model.addConstr(constr, name=cons_name)
    
    def _constraint_part_purchase(self) -> None:
        """ Limit the total number of parts purchased.

        A simple balance for the allowed total across all part-color combinations.
        """
        constr = (
            sum(
                self.iv_purchase[p, c] for (p, c) in s_part_in_color
            ) <= self.p_max_parts_purchased
        )
        self.model.addConstr(constr, name="c_total_purchase_limit_balance")
    
    def _objective_function(self) -> None:
        """ Modified objective function.
        
        A second term that reflects the incurred cost of purchasing parts.
        """
        obj = sum(
            self.p_set_num_value[s] * self.bv_select_set[s]
            for s in self.s_lego_set
        ) - self.p_part_cost*sum(
            self.iv_purchase[p, c]
            for (p, c) in self.s_part_in_color
        )
        self.model.setObjective(obj, GRB.MAXIMIZE)
    
    def define_model(self) -> None:
        """ Modified model definition method.

        Since we are including new variables, constraints and a modified
        objective function, we need to override the base class method with
        a sequence of method calls that reflects the new logic.
        """
        self._binary_variables()
        self._integer_variables()
        self._constraint_part_inventory()
        self._constraint_part_purchase()
        self._objective_function()
        self.model.update()


%md
### Build & Solve Part Purchase Model

The model is now complete. The solver will search the feasible space to build the most valuable collection of complete sets based on the perturbed inventory and the allowed part purchase budget from the parts corresponding to the 203 starting Star Wars sets.

To make things interesting, let's solve the model for a range of part budgets. This analysis demonstrates the ability of optimization to provide answers to what-if scenarios. This is both demonstrated by:
- Including additional logic that models the ability to purchase parts
- Experiment with the richer modified model by adjusting the parameters that define our part purchase logic

We don't need to re-instantiate the model for each point in the experiment. We simply redefine the class parameter across the experimental space and call the `define_model` and `solve_model` methods as we iterate to obtain the desired solutions.

We store the solutions for each experimental point in order to visualize the results.

In [0]:
p_part_cost = 10 # Assumed cost for purchasing a part (constant across all parts)
purchase_solutions_df = pd.DataFrame(columns=['Optimal Value', 'Max Parts Purchased', 'Unused Inventory Parts Total'])

# A list of scenarios for different values of maximum number of parts to purchase
p_max_parts_purchased_set = list(range(0, 191, 10)) + list(range(200, 2101, 100))
purchase_model = PartPurchaseLegoModel(
    s_lego_set,
    s_part_in_set,
    s_part_in_color,
    p_part_quantity_in_set,
    p_part_quantity_available,
    p_set_num_value,
    0,
    p_part_cost,
)

with mlflow.start_run(run_name="PartPurchaseLegoModel Part Budget Sensitivity") as run:
    # We solve for all scenarios of limit on maximum number of parts purchased ina  for loop
    for i, p_max_parts_purchased in enumerate(p_max_parts_purchased_set):   
        purchase_model.p_max_parts_purchased = p_max_parts_purchased
        purchase_model.define_model()
        purchase_model.solve_model()
        purchase_model.write_solution()
        unused_inventory_parts_total = purchase_model.df_remaining_parts_data.Quantity.sum()
        purchase_solutions_df.loc[i] = [purchase_model.optimal_value, p_max_parts_purchased, unused_inventory_parts_total]
    ax_cost = purchase_solutions_df.plot(x='Max Parts Purchased', y='Optimal Value', figsize=(16, 4))
    ax_cost.grid(axis='y')

    ax_parts = purchase_solutions_df.plot(x='Max Parts Purchased', y='Unused Inventory Parts Total', figsize=(16, 4))
    ax_parts.grid(axis='y')

    # Store the dataframe to MLflow
    purchase_solutions_df.to_csv("purchase_parts_sensitivity_solutions.csv")
    mlflow.log_artifact("./purchase_parts_sensitivity_solutions.csv")

    # Store the plots to MLflow
    mlflow.log_figure(ax_cost.get_figure(), "purchase_model_cost_plot.png")
    mlflow.log_figure(ax_parts.get_figure(), "purchase_model_parts_plot.png")

The part purchase option recovers value optimally according to the part purchase budget. Gurobi has selected the precise set that maximizes the overall collection value at each specified point, with a global optimality guarantee, while respecting the allowed budget for parts purchase. As shown by the figures above, one can adjust the maximum number of parts to be purchased to a high enough value to recover all the initial sets (e.g. `p_max_parts_purchased = 2500`). However, we can't recover the original value as the modified objective takes into account the incurred cost of parts purchase. The Number of unused pieces decreases at a rate the almost mirrors the increase of value as more parts are allowed. Finally, note that the extended model formulation is equivalent to the original base model formulation if the part budgets is set to zero. 

This exercise demonstrates how practical considerations can be incrementally incorporated into an optimization model. This enables a thorough analysis of the trade-offs that organizations and decision-makers encounter to make more informed and effective choices.

Based on our analysis of varying parts budgets, we decide to store a final run with an allowance for purchasing 300 parts. Based on the plots above, we can expect to recover more than 30,000 in value while reducing the unused inventory by more than 5000 bricks with this selection.

In [0]:
purchase_model.p_max_parts_purchased = 300

# Start an MLflow run
with mlflow.start_run(run_name=f"PartPurchaseLegoModel Part Budget {purchase_model.p_max_parts_purchased}") as run:
    purchase_model.define_model()
    purchase_model.solve_model()
    purchase_model.write_solution()
    
    # Log parameters and metrics
    mlflow.log_param("p_max_parts_purchased", purchase_model.p_max_parts_purchased)
    mlflow.log_metric("optimal_value", purchase_model.optimal_value)
    mlflow.log_metric("total_inventory_unused_bricks", purchase_model.df_remaining_parts_data.Quantity.sum())

In [0]:
# Dispose purchase model instance
purchase_model.model.dispose()
purchase_model.env.dispose()

### What's Next?

Through this series of notebooks, we have:
- introduced key concepts of mathematical optimization
- developed a mathematical optimization model for a LEGO assortment problem
- implemented the model and solved it with Gurobi in Databricks
- demonstrated scalability of mathematical optimization by applying it on a large dataset
- showcased the value of optimization by comparing results of mathematical optimization with Gurobi against a greedy heuristic
- demonstrated scenario analysis with a variant of our original mathematical model

This concludes our initial deep dive into mathematical optimization. Please keep an eye on the Databricks website for our upcoming accelerator on Assortment Optimization which will demonstrate how to tackle a complex, large scale and real-world application of mathematical optimization applicable for many retail and consumer goods organizations.

### Authors
These notebooks and the associated [blog](hyperlink when link is ready) were jointly authored by [Juan Morinelli](https://www.aimpointdigital.com/team/juan-morinelli) (Aimpoint Digital), Linlin Yang (Aimpoint Digital), Peyman Mohajerian (Databricks) and Bryan Smith (Databricks).

### About Aimpoint Digital
Aimpoint Digital is a market-leading analytics firm at the forefront of solving the most complex business and economic challenges through data and analytical technology. From integrating self-service analytics to implementing AI at scale and modernizing data infrastructure environments, Aimpoint Digital operates across transformative domains to improve the performance of organizations. Learn more by visiting: https://www.aimpointdigital.com/